# Atmospheric Heat Content (AHC)
$$ \mathrm{AHC} = \frac{1}{g} \int \int \int_{0}^{1000hPa} T(p, x, y) dp dx dy$$

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from maps import map_robinson, map_eq_earth
from GMST import GMST_timeseries, GMST_regression, atm_heat_content, GMST_GISTEMP
from paths import path_results, path_samoc, path_data
from plotting import shifted_color_map, discrete_cmap
from constants import abs_zero, cp_air
from timeseries import IterateOutputCESM
from xr_integrate import xr_surf_mean, xr_zonal_mean
from xr_DataArrays import xr_AREA

In [ ]:
ds = xr.open_dataset(file_ex_atm_ctrl)
dp = (ds.ilev-ds.ilev.shift(ilev=1)).dropna(dim='ilev')
dp = dp.drop(['ilev'])
dp = dp.rename({'ilev':'lev'})
dp = dp.assign_coords(lev=ds.lev)

In [ ]:
%%time
# 100 sec
for run in ['ctrl', 'rcp']:
    print(run)
    for i, (y,m,s) in enumerate(IterateOutputCESM(domain='atm', run=run, tavg='yrly', name='T_T850_U_V')):
        ds = xr.open_dataset(s, decode_times=False)
        da = atm_heat_content(ds, dp, AREA)    
        if i==0:
            da_new = da
        else: 
            da_new = xr.concat([da_new, da], dim='time')
    da_new.to_netcdf(f'{path_results}/GMST/AHC_yrly_{run}.nc')

In [ ]:
AHC_ctrl  = xr.open_dataarray(f'{path_results}/GMST/AHC_yrly_ctrl.nc', decode_times=False)
AHC_rcp   = xr.open_dataarray(f'{path_results}/GMST/AHC_yrly_rcp.nc' , decode_times=False)
dAHC_ctrl = (AHC_ctrl - AHC_ctrl.shift(time=1)).dropna(dim='time')
dAHC_rcp  = (AHC_rcp  - AHC_rcp .shift(time=1)).dropna(dim='time')

In [ ]:
AHC_ctrl[101].time/365

In [ ]:
AHC_0 = AHC_ctrl[101]

f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.axhline(0, c='k', lw=.5)
plt.axvline(200, c='g', lw=.5)
plt.plot(AHC_ctrl.time/365    , (AHC_ctrl-AHC_0)/1e21)
plt.plot(AHC_rcp .time/365+200, (AHC_rcp -AHC_0)/1e21)
plt.ylabel('AHC anomaly [ZJ]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.tight_layout()

f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.axhline(0, c='k', lw=.5)
plt.axvline(200, c='g', lw=.5)
plt.plot(dAHC_ctrl.time/365    , dAHC_ctrl/1e21)
plt.plot(dAHC_rcp .time/365+300, dAHC_rcp /1e21)
plt.ylabel('$\Delta$AHC [ZJ/yr]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/dAHC')

In [ ]:
dGMST_ctrl = (gmst_ctrl.GMST - gmst_ctrl.GMST.shift(time=1)).dropna(dim='time')
dGMST_rcp  = (gmst_rcp .GMST - gmst_rcp .GMST.shift(time=1)).dropna(dim='time')

In [ ]:
plt.axhline(0, c='k', lw=.5)
plt.axvline(0, c='k', lw=.5)
plt.scatter(dGMST_ctrl-dGMST_ctrl.mean(), dAHC_ctrl-dAHC_ctrl.mean())
plt.scatter(dGMST_rcp -dGMST_rcp .mean(), dAHC_rcp -dAHC_rcp .mean())

as expected, positive correlation between $\Delta$AHC and $\Delta$GMST